In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install torchvision
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import datasets
import transformers
from transformers import AutoTokenizer, AlbertForSequenceClassification, AdamW
	
#from transformers import AlbertTokenizer, AlbertForSequenceClassification, AdamW
from sklearn.metrics import classification_report
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda


In [ ]:
# Load the dataset
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary', split='train[:1000]')   
#dataset = load_dataset(\https://huggingface.co/datasets/ucberkeley-dlab/measuring-hate-speech\)
# Split the dataset into train and test sets
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
# Define a function for cleaning and preprocessing the text
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Remove stop words and lemmatize the tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    # Join the tokens back into a string
    text = ' '.join(tokens)
    
    return text

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

	
train_data['label'] = [0 if label <0.5 else 1 for label in train_data['hate_speech_score']]
test_data['label'] = [0 if label <0.5 else 1 for label in test_data['hate_speech_score']]
# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)
	
	#tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
	#model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)
	
	# Tokenize the input text
train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of

In [ ]:
# Convert the tokenized input into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']), torch.tensor(train_encodings['attention_mask']), torch.tensor(list(train_data['label'])))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']), torch.tensor(test_encodings['attention_mask']), torch.tensor(list(test_data['label'])))
	
# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()
	


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Train the model
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
for epoch in range(3):
  for batch in train_loader:
    optimizer.zero_grad()
    input_ids, attention_mask, labels = tuple(t.to('cpu') for t in batch)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

In [ ]:
# Evaluate the model on the test set
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)
predictions = []
with torch.no_grad():
  for batch in test_loader:
    input_ids, attention_mask, _ = tuple(t.to('cpu') for t in batch)
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    predictions.extend(predicted_labels.cpu().detach().numpy())


In [ ]:
# Print the classification report
target_names = ['non-hate', 'hate']
print(classification_report(list(test_data['label']), predictions, target_names=target_names))
# Print the accuracy score
accuracy = accuracy_score(list(test_data['label']), predictions)
print('Accuracy: {:.2f}%'.format(accuracy*100))


              precision    recall  f1-score   support

    non-hate       0.83      0.85      0.84       156
        hate       0.42      0.39      0.40        44

    accuracy                           0.75       200
   macro avg       0.63      0.62      0.62       200
weighted avg       0.74      0.75      0.75       200

Accuracy: 75.00%


# New section